In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!nvidia-smi
!cat /proc/cpuinfo
!rm -rf sample_data

!git clone --recursive https://github.com/Microsoft/LightGBM.git
!cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Sat May  1 05:38:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!cp /content/drive/MyDrive/0403.zip 0403.zip
!unzip /content/drive/MyDrive/0403.zip

Archive:  /content/drive/MyDrive/0403.zip
  inflating: 0403/5cd56b6be2acfd2d33b59d1f_test.csv  
  inflating: 0403/5cd56b6ae2acfd2d33b59c90_test.csv  
  inflating: 0403/5cd56b6be2acfd2d33b59d1f_train.csv  
  inflating: 0403/5cd56b6ee2acfd2d33b5a247_test.csv  
  inflating: 0403/5cd56b7de2acfd2d33b5c14b_train.csv  
  inflating: 0403/5cd56b6ae2acfd2d33b59c90_train.csv  
  inflating: 0403/5cd56b7de2acfd2d33b5c14b_test.csv  
  inflating: 0403/5cd56b6ee2acfd2d33b5a247_train.csv  
  inflating: 0403/5cd56b6fe2acfd2d33b5a386_test.csv  
  inflating: 0403/5cd56b6fe2acfd2d33b5a386_train.csv  
  inflating: 0403/5cd56b6ae2acfd2d33b59ccc_train.csv  
  inflating: 0403/5cd56b5ae2acfd2d33b58549_test.csv  
  inflating: 0403/5cd56b5ae2acfd2d33b58546_train.csv  
  inflating: 0403/5cd56b6ae2acfd2d33b59ccc_test.csv  
  inflating: 0403/5cd56b5ae2acfd2d33b58546_test.csv  
  inflating: 0403/5d27099f03f801723c32511d_test.csv  
  inflating: 0403/5cd56b5ae2acfd2d33b58549_train.csv  
  inflating: 0403/5da1383b4db8ce

In [5]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob

from sklearn.model_selection import KFold
import lightgbm as lgb

import psutil
import random
import os
import time
import sys
import math
from contextlib import contextmanager


N_SPLITS = 10
SEED = 42

@contextmanager
def timer(name: str):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    try:
        yield
    finally:
        m1 = p.memory_info()[0] / 2. ** 30
        delta = m1 - m0
        sign = '+' if delta >= 0 else '-'
        delta = math.fabs(delta)
        print(f"[{m1:.1f}GB({sign}{delta:.1f}GB): {time.time() - t0:.3f}sec] {name}", file=sys.stderr)


def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]





set_seed(SEED)

feature_dir = './*/'
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))

#parameters for the three LightGBM models
lgb_params = {'objective': 'root_mean_squared_error',
              'boosting_type': 'gbdt',
              'n_estimators': 50000,
              'learning_rate': 0.05,
              'num_leaves': 90,
              'device': 'gpu',
              'colsample_bytree': 0.4,
              'subsample': 0.6,
              'subsample_freq': 2,
              'bagging_seed': SEED,
              'reg_alpha': 2,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }

lgb_f_params = {'objective': 'multiclass',
                'boosting_type': 'gbdt',
                'n_estimators': 50000,
                'learning_rate': 0.05,
                'num_leaves': 90,
                'device': 'gpu',
                'colsample_bytree': 0.4,
                'subsample': 0.6,
                'subsample_freq': 2,
                'bagging_seed': SEED,
                'reg_alpha': 10,
                'reg_lambda': 2,
                'random_state': SEED,
                'n_jobs': -1
                }

['./0403/5cd56b5ae2acfd2d33b58544_train.csv', './0403/5cd56b5ae2acfd2d33b58546_train.csv', './0403/5cd56b5ae2acfd2d33b58548_train.csv', './0403/5cd56b5ae2acfd2d33b58549_train.csv', './0403/5cd56b5ae2acfd2d33b5854a_train.csv', './0403/5cd56b6ae2acfd2d33b59c90_train.csv', './0403/5cd56b6ae2acfd2d33b59ccb_train.csv', './0403/5cd56b6ae2acfd2d33b59ccc_train.csv', './0403/5cd56b6be2acfd2d33b59d1f_train.csv', './0403/5cd56b6ee2acfd2d33b5a247_train.csv', './0403/5cd56b6fe2acfd2d33b5a386_train.csv', './0403/5cd56b7de2acfd2d33b5c14b_train.csv', './0403/5d27075f03f801723c2e360f_train.csv', './0403/5d27096c03f801723c31e5e0_train.csv', './0403/5d27097f03f801723c320d97_train.csv', './0403/5d27099f03f801723c32511d_train.csv', './0403/5d2709a003f801723c3251bf_train.csv', './0403/5d2709b303f801723c327472_train.csv', './0403/5d2709bb03f801723c32852c_train.csv', './0403/5d2709c303f801723c3299ee_train.csv', './0403/5d2709d403f801723c32bd39_train.csv', './0403/5da1382d4db8ce0c98bbe92e_train.csv', './0403/5

# Calculate all data

In [7]:
from sklearn.metrics import mean_squared_error

score_df = pd.DataFrame()
oof = list()
predictions = list()
for n_files, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0)
    
    test_data = pd.read_csv(test_files[n_files], index_col=0)

    oof_x, oof_y, oof_f = np.zeros(data.shape[0]), np.zeros(data.shape[0]), np.zeros(data.shape[0])
    # oof_x, oof_y, oof_f is the np.zeros to store X, Y and Floor result
    preds_x, preds_y = 0, 0
    preds_f_arr = np.zeros((test_data.shape[0], N_SPLITS))
    print("Starting to train: ", os.path.basename(file))
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(data.iloc[:, :-4])):
        X_train = data.iloc[trn_idx, :-5] # All wifi signal info
        y_trainx = data.iloc[trn_idx, -5] # the location X to be predicted
        y_trainy = data.iloc[trn_idx, -4] # the location Y to be predicted
        y_trainf = data.iloc[trn_idx, -3] # the location's floor

        X_valid = data.iloc[val_idx, :-5] # All wifi signal info in KFold validation dataset
        y_validx = data.iloc[val_idx, -5] # the location X to be predicted in KFold validation dataset
        y_validy = data.iloc[val_idx, -4] # the location Y to be predicted in KFold validation dataset
        y_validf = data.iloc[val_idx, -3] # the location's floor in KFold validation dataset

        modelx = lgb.LGBMRegressor(**lgb_params) #Model selected, which is LightGBM
        with timer("fit X"):
            modelx.fit(X_train, y_trainx,
                       eval_set=[(X_valid, y_validx)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=20
                       ) #Model 1, used to predit X location only

        modely = lgb.LGBMRegressor(**lgb_params)
        with timer("fit Y"):
            modely.fit(X_train, y_trainy,
                       eval_set=[(X_valid, y_validy)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=20
                       ) #Model 2, used to predit Y location only

        modelf = lgb.LGBMClassifier(**lgb_f_params)
        with timer("fit F"):
            modelf.fit(X_train, y_trainf,
                       eval_set=[(X_valid, y_validf)],
                       eval_metric='multi_logloss',
                       verbose=False,
                       early_stopping_rounds=20
                       ) #Model 3, used to predit Floor location only

        oof_x[val_idx] = modelx.predict(X_valid)
        oof_y[val_idx] = modely.predict(X_valid)
        oof_f[val_idx] = modelf.predict(X_valid).astype(int)

        preds_x += modelx.predict(test_data.iloc[:, :-5]) / N_SPLITS 
        #Average the result of each folds, divide every result by n_splits, and add them together
        preds_y += modely.predict(test_data.iloc[:, :-5]) / N_SPLITS
        #Average the result of each folds, divide every result by n_splits, and add them together
        preds_f_arr[:, fold] = modelf.predict(test_data.iloc[:, :-5]).astype(int)
        #The floor is dealt differently,   

        score = comp_metric(oof_x[val_idx], oof_y[val_idx], oof_f[val_idx],
                            y_validx.to_numpy(), y_validy.to_numpy(), y_validf.to_numpy())
        # the error in current KFold
        print(f"fold {fold}: mean position error {score}")


    # Evaluate mean position error of each file
    print("*+"*40)
    print(f"file #{n_files}, shape={data.shape}, name={os.path.basename(file)}")
    score = comp_metric(oof_x, oof_y, oof_f,
                        data.iloc[:, -5].to_numpy(), data.iloc[:, -4].to_numpy(), data.iloc[:, -3].to_numpy())
    #Calculate the score/error
    oof.append(score)
    print(f"mean position error {score}")
    print("*+"*40)

    preds_f_mode = stats.mode(preds_f_arr, axis=1)
    preds_f = preds_f_mode[0].astype(int).reshape(-1)
    #The floor is dealt differently, vote to choose the most appeared floor number in all folds
    test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
    ######
    #Stack all the predictions in one testing file together, and transpose to vertical
    predictions.append(test_preds)
    # print(test_preds)

    column_num = len(test_data.columns)
    test_data = test_data.reset_index()

    test_data.insert(column_num, 'pred_x', test_preds[1], True)
    test_data.insert(column_num+1, 'pred_y', test_preds[2], True)
    test_data.insert(column_num+2, 'pred_f', test_preds[0], True) 

    building_name = file[7:-10] + '.csv'

    test_data.to_csv(building_name)

print("*+"*40)


Starting to train:  5cd56b5ae2acfd2d33b58544_train.csv


[0.3GB(+0.1GB): 7.057sec] fit X
[0.3GB(+0.0GB): 2.096sec] fit Y
[0.3GB(+0.0GB): 7.779sec] fit F


fold 0: mean position error 2.6661671129425537


[0.3GB(+0.0GB): 1.666sec] fit X
[0.3GB(+0.0GB): 2.859sec] fit Y
[0.3GB(+0.0GB): 3.346sec] fit F


fold 1: mean position error 2.2981185355325886


...

fold 7: mean position error 3.5136253837335603


[2.6GB(+0.0GB): 45.254sec] fit X
[2.6GB(+0.0GB): 31.016sec] fit Y
[2.6GB(+0.0GB): 96.833sec] fit F


fold 8: mean position error 3.485730188497854


[2.6GB(+0.0GB): 49.823sec] fit X
[2.6GB(+0.0GB): 55.991sec] fit Y
[2.6GB(+0.0GB): 226.855sec] fit F


fold 9: mean position error 3.5700653752869034
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
file #24, shape=(11361, 1365), name=5da958dd46f8266d0737457b_train.csv
mean position error 3.523361857568685
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
6.0006500043331075
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+


In [23]:
pred_files = glob.glob('/content' + '/*.csv')

In [31]:
# extract true and predicted x, y and f
all_pred = pd.DataFrame()

for file_name in pred_files:
  file = pd.read_csv(file_name)

  all_pred = pd.concat([all_pred, file.iloc[:, -8:]])


In [32]:
all_pred

,x,y,f,path,pred_x,pred_y,pred_f,timestamp
0,81.543450,47.366474,8,5cfce0775163d8000807bf07,76.198427,63.352369,8.0,1560075841746
1,81.543450,47.366474,8,5cfce0775163d8000807bf07,83.091023,45.971878,8.0,1560075843593
2,96.410450,55.596020,8,5cfce0775163d8000807bf07,89.394663,47.271994,8.0,1560075844985
3,96.410450,55.596020,8,5cfce0775163d8000807bf07,91.400078,50.210441,8.0,1560075845701
4,96.410450,55.596020,8,5cfce0775163d8000807bf07,97.203798,55.260943,8.0,1560075849217
...,...,...,...,...,...,...,...,...
1244,9.271838,144.471220,-1,5cf3a0505b96c60008d35b1c,13.885404,125.580031,-1.0,1559470102975
1245,9.271838,144.471220,-1,5cf3a0505b96c60008d35b1c,13.885404,125.580031,-1.0,1559470103595
1246,9.271838,144.471220,-1,5cf3a0505b96c60008d35b1c,13.885404,125.580031,-1.0,1559470105536
1247,9.271838,144.471220,-1,5cf3a0505b96c60008d35b1c,13.885404,125.580031,-1.0,1559470107390


In [33]:
# calculate MSE
comp_metric(all_pred['pred_x'], all_pred['pred_y'], all_pred['pred_f'], all_pred['x'], all_pred['y'], all_pred['f'])

4.512068593838267